# Ouverture du fichier
Vous pouvez cloner le fichier par contre il faudra que vous ajoutiez le fichier challenge_set.json qui est trop volumineux pour github. Celui-ci se trouve dans le dossier `spotify_million_playlist_dataset_challenge` qui lui même est donné en suivant les liens qui se trouvent dans les consignes. Une fois que vous avez mis ce fichier dans votre dossier avec le code. tout devrait run sans problème
Faites un `Ctrl+F` avec "ici" pour trouver tous les endroits où il y a un petit bug ou une optimisation à faire.  

---

## Done

Les df créés sont les suivants :

- **`df_playlists`** : contient toutes les playlists avec leurs noms et leurs paramètres.  
  La colonne `tracks` contient toutes les informations sur les musiques de la playlist, mais dans un format très « dirty ».

- **`df_tracks`** : correspond à la colonne `playlists["tracks"]`, un peu nettoyée.  
  Une même musique peut apparaître plusieurs fois.

- **`df_tracks_vectors`** : contient `track_id` et le vecteur `list_pid`de dimension 10 000 permettant de calculer la similarité entre deux musiques. Là par contre chaque musique aparait qu'une seule fois

---

## To do

- Vérifier ou optimiser le code.  
  Normalement, la question 3 est résolue.  
  Il serait intéressant de créer une fonction qui accepte **l'artiste et le titre** pour être plus user-friendly, car la fonction actuelle n'accepte que des `track_id`.  
  Il faudra préparer des exemples et tester avec potentiellement des musiques que l'on connait.
- Un des "ici" correspond à un commentaire où j'essaie d'ajouter le nom de playlist dans une liste, mais cela ne fonctionne pas. Je ne comprends pas trop pourquoi, surtout que ça marche avec le `pid`. De facon dirty peut aller rechercher l'information après dans les autre df mais ce serait quand même moins optimal bien que fonctionel
- Faire des statistiques descriptives pour la question 2.
- Esquisser une formule pour la question 4
- Esquisser une formule pour la question 5


In [ ]:
import pandas as pd
import numpy as np
import sys
import json
import math
from itertools import combinations

In [4]:
here_Path = sys.path[0]
data_Path = here_Path + "\\challenge_set.json"
with open(data_Path, 'r', encoding='utf8') as f:
    data = json.load(f)

# Extraire uniquement la liste des playlists
playlists = data["playlists"]

# Transformer en DataFrame
df_playlists = pd.DataFrame(playlists)
df_playlists["name"] = df_playlists["name"].astype(str)
df_playlists["pid"] = df_playlists["pid"].astype(str)
df_playlists.info()

FileNotFoundError: [Errno 2] No such file or directory: '/opt/anaconda3/envs/environnement_test/lib/python313.zip\\challenge_set.json'

In [15]:
from pathlib import Path
import json
import pandas as pd

base_dir = Path.cwd()  
data_path = base_dir / "spotify_million_playlist_dataset_challenge" / "challenge_set.json"

with data_path.open("r", encoding="utf-8") as f:
    data = json.load(f)

playlists = data["playlists"]

df_playlists = pd.DataFrame(data["playlists"], dtype=str)
df_playlists["name"] = df_playlists["name"].astype(str)
df_playlists["pid"] = df_playlists["pid"].astype(str)
df_playlists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          10000 non-null  object
 1   num_holdouts  10000 non-null  object
 2   pid           10000 non-null  object
 3   num_tracks    10000 non-null  object
 4   tracks        10000 non-null  object
 5   num_samples   10000 non-null  object
dtypes: object(6)
memory usage: 468.9+ KB


In [21]:
df_playlists.shape

(10000, 6)

## On construit ici une df des tracks cleaned dans le sens ou les id sont débarrasé de *spotify:name:*
    - pid = playlist id

In [16]:
# ici le hardcode est pas propre mais bon
all_tracks = []
df_tracks = pd.DataFrame([])
for pl in playlists:
    for tr in pl["tracks"]:
        track_info = tr.copy()  # pour ne pas modifier l'objet original
        #track_info["pname"] = pl["name"] ici je ne sais pas pourquoi mais il veut pas ajouter le nom de la playlist
        track_info["pid"] = pl["pid"]
        all_tracks.append(track_info)
df_tracks = pd.DataFrame(all_tracks)
# ici y'a moyen d'optimiser la boucle en la rentrant au dessus
for i in ( "track", "artist", "album"):
    df_tracks[i+"_id"] = df_tracks[i+"_uri"].str.split(":").str[-1]
df_tracks.drop(columns=["track_uri", "artist_uri", "album_uri"], inplace=True)
#df_tracks.head()

A priori, si $\text{track\_id}_i = \text{track\_id}_j$ :  

$$
\text{artist\_id}_i = \text{artist\_id}_j, \quad
\text{album\_id}_i = \text{album\_id}_j
$$

On peut donc chercher par `track_id` unique et créer une liste pour le `pid`. Ainsi, *list_pid* sera un vecteur de dimension $( \text{ndim} = 10 000$. Seules les composantes non nulles sont conservées et valent 0 ou 1 si elles sont présentes dans la playlist.  
Plutôt que de conserver les 0, on conserve uniquement les indices des playlists dans lesquelles la musique apparaît.  
(On pourrait éventuellement retirer les playlists avec zéro track.)

La similarité entre deux musiques peut alors se calculer de la façon suivante :  

$$
\langle a, b \rangle = \frac{|a \cap b|}{\sqrt{\text{len}(a) \cdot \text{len}(b)}} 
$$


In [19]:
df_tracks.shape

(281000, 9)

In [22]:
df_tracks_vectors = (
    df_tracks.groupby("track_id", as_index=True).agg({
        "pid": list
    })
)
df_tracks_vectors.rename(columns={"pid": "list_pid"}, inplace=True)
df_tracks_vectors["pid_count"] = df_tracks_vectors["list_pid"].apply(len)
df_tracks_vectors = df_tracks_vectors.sort_values("pid_count", ascending=False)
df_tracks_vectors.head()


,list_pid,pid_count
track_id,,
7yyRTcZmCiyzzJlNzGC9Ol,"[1000791, 1000859, 1001452, 1001497, 1001589, ...",226
5dNfHmqgr128gMY2tc5CeJ,"[1001292, 1002975, 1003413, 1003871, 1004619, ...",222
5XJJdNPkwmbUwE79gv0NxK,"[1001692, 1007829, 1007941, 1008258, 1008788, ...",215
62vpWI1CHwFy7tMIcSStl8,"[1000789, 1000801, 1002510, 1003021, 1003028, ...",211
7BKLCZ1jbUBVqRi2FVlTVw,"[1000430, 1000859, 1001897, 1002348, 1002729, ...",206


In [29]:
df_tracks_vectors.shape

(66243, 2)

In [23]:
def similar_tracks(track_id1,track_id2):
    try:
        list1 = df_tracks_vectors.loc[track_id1, "list_pid"]
        list2 = df_tracks_vectors.loc[track_id2, "list_pid"]
    except KeyError:
        return None
    # transformer en sets pour calculer l'intersection
    set1, set2 = set(list1), set(list2)
    n_common = len(set1 & set2)
    
    if n_common == 0:
        return 0.0
    # ici à priori pas de division par zéro car len > 0 mais bon
    return n_common / math.sqrt(len(set1) * len(set2))

# exemple d'utilisation de similar track

In [ ]:
print(f"2 musiques similaires : {similar_tracks('66U0ASk1VHZsqIkpMjKX3B', '66U0ASk1VHZsqIkpMjKX3B'):.3f}")
print(f"2 musiques du meme album  : {similar_tracks('35kahykNu00FPysz3C2euR', '3G6hD9B2ZHOsgf4WfNu7X1'):.3f}")
print(f"2 musiques les plus partagées  : {similar_tracks('7yyRTcZmCiyzzJlNzGC9Ol', '5dNfHmqgr128gMY2tc5CeJ'):.3f}")

2 musiques similaires : 1.000
2 musiques du meme album  : 0.297
2 musiques du meme album  : 0.096


# Q.4

L'idée est d'adopter la même logique que pour la question 3 sauf qu'ici l'idée n'est plus de représenter chaque track selon la liste de ses playlists mais plutôt de représenter chaque playlist par sa liste de tracks. L'espace de représentation est donc le nombre de tracks. 

Pour deux playlists P1 et P2 : 
+ T(P1) = l’ensemble des track_id dans la playlist 1
+ T(P2) = l'ensemble des track_id de la playlist 2 

On peut définir :

$$
\langle P1, P2 \rangle = \frac{|T(P1) \cap T(P2)|}{\sqrt{\text{len}(TP(1)) \cdot \text{len}(TP(2))}} 
$$

Donc, deux playlists sont similaires si elles partagent beaucoup de morceaux, proportionnellement à leur longueur. 

In [24]:
df_playlists['pid']

0       1000002
1       1000003
2       1000004
3       1000006
4       1000007
         ...   
9995    1006767
9996    1006771
9997    1006773
9998    1006775
9999    1006778
Name: pid, Length: 10000, dtype: object

In [ ]:
# on passe à une représentation une playlist -> liste de tracks. De la même manière qu'avant, chaque ligne représente une playliste et la liste de ses musiques 

df_playlist_vectors = (
    df_tracks.groupby("pid", as_index=True)
             .agg({"track_id": list})
             .rename(columns={"track_id": "list_track_id"})
)
df_playlist_vectors["track_count"] = df_playlist_vectors["list_track_id"].apply(len)

In [31]:
df_playlist_vectors.head()

,list_track_id,track_count
pid,,
1000000,"[66U0ASk1VHZsqIkpMjKX3B, 5MhsZlmKJG6X5kTHkdwC4...",5
1000001,"[3kpM8OxeMaaAWI9pErdj1S, 4sj8qFcEDnRPNv3tBbUVU...",25
1000009,"[7rPLZ8Krm6CZIbraFUlnWZ, 4LloVtxNZpeh7q7xdi1DQ...",25
1000016,"[6WQLkih8nE0JdUCEyLaGnQ, 37sINbJZcFdHFAsVNsPq1...",5
1000020,"[4PvD06Pmbm2rHG2JjSlElF, 57nNNkgk768QVXq3uHxu5...",5


In [27]:
df_playlist_vectors.head()
df_playlist_vectors.shape

(9000, 2)

In [ ]:
nb_playlists_total = df_playlists["pid"].nunique()
nb_playlists_non_vides = df_tracks["pid"].nunique()
nb_playlists_vides = nb_playlists_total - nb_playlists_non_vides
print(nb_playlists_vides) #df_playlist_vectors ne contient que les playlists non-vides 

1000


In [30]:
# on crée ensuite une fonction de similarité semblable à celle créée pour les tracks 
def similar_playlists(pid1, pid2):
    try:
        list1 = df_playlist_vectors.loc[pid1, "list_track_id"] # on récupère la liste des morceaux associés à la playlist pid1
        list2 = df_playlist_vectors.loc[pid2, "list_track_id"] # on récupère la liste des morceaux associés à la playlist pid2
    except KeyError:
        return None

    set1, set2 = set(list1), set(list2) # on transforme en set pour n'avoir que des éléments uniques 
    n_common = len(set1 & set2)
    if n_common == 0:
        return 0.0

    return n_common / math.sqrt(len(set1) * len(set2))

In [38]:
# helper fait avec chat pour avoir le top des playlists qui partagent le plus de sons pour faire quelques checks

def top_common_playlists(limit=10):
    # convertit une fois pour toutes en sets
    playlist_sets = {
        pid: set(row["list_track_id"])
        for pid, row in df_playlist_vectors.iterrows()
    }

    scores = []
    for (pid1, set1), (pid2, set2) in combinations(playlist_sets.items(), 2):
        common = len(set1 & set2)
        if common > 0:
            scores.append((common, pid1, pid2))
    scores.sort(reverse=True)
    return scores[:limit]

pairs = top_common_playlists(5)
print(pairs)  # listes de (nb commun, pid1, pid2)

[(88, 1014985, 1028896), (59, 1028896, 1037525), (59, 1020013, 1024428), (58, 1023697, 1037525), (57, 1014985, 1037525)]


In [40]:
print(f"2 playlists identiques : {similar_playlists(1000000, 1000000):.3f}") # doit renvoyer 1
print(f"2 playlists avec un morceau commun : {similar_playlists(1036931, 1037176):.3f}") #contiennent toutes les deux Stays in Mexico 
print(f"2 playlists les plus semblables (top 1) : {similar_playlists(1014985, 1028896):.3f}") 
print(f"2 playlists les plus semblables (top 2) : {similar_playlists(1023697, 1037525):.3f}") #contiennent toutes les deux Stays in Mexico 

2 playlists identiques : 1.000
2 playlists avec un morceau commun : 0.142
2 playlists les plus semblables (top 1) : 0.880
2 playlists les plus semblables (top 2) : 0.580


In [43]:
df_playlist_vectors.loc[[1023697,1037525]]

,list_track_id,track_count
pid,,
1023697,"[3wkKkFAtYSTRwqOydW6T0I, 5rgy6ghBq1eRApCkeUdJX...",100
1037525,"[4P7VFiaZb3xrXoqGwZXC3J, 0DdpxWfVvUGgkJv5536ti...",100


Prochaines étapes : 
+ pondérer les tracks par leur popularité -> pour que les tracks populaires fassent moins exploser le score de similarité entre des playlists. Avec un truc comme # de playlists dans lesquelles apparaît le track / nombre total de playlists. L'intuition serait qu'un mocreau rare partagé entre deux playlists a plus de poids dans la définition de similarité qu'un morceau qui est très connu. 
+ introduire un score selon le nom des artistes partagés ou albums partagés même si ce ne sont pas les mêmes musiques ? Mais un artiste peut avoir plusieurs styles et idem pour un album
+ similarité dans le nom des playlists -> similarité : sport, muscu, course à pied etc/ relaxation, détente, etc./ étude, focus, etc. J'ai vu certaines playlists appelées workout. 